In [1]:
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.llms import CTransformers


In [7]:
#Extract data from the PDF
def load_pdf(data):
    loader = DirectoryLoader(data,
                    glob="*.pdf",
                    loader_cls=PyPDFLoader)
    
    documents = loader.load()

    return documents

In [8]:
extracted_data = load_pdf("D:/Data/GenAI Ineuron/Day12/data/")

In [12]:
# extracted_data

In [10]:
#Create text chunks
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 20)
    text_chunks = text_splitter.split_documents(extracted_data)

    return text_chunks

In [11]:
text_chunks = text_split(extracted_data)
print("length of my chunk:", len(text_chunks))

length of my chunk: 7020


In [2]:
#download embedding model
def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

In [3]:
embeddings = download_hugging_face_embeddings()

d:\anaconda\envs\mchatbot\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [31]:
# #Initializing the Pinecone
# pinecone.init(api_key=PINECONE_API_KEY)
# index_name="testing"
# #Creating Embeddings for Each of The Text Chunks & storing
# docsearch=Pinecone.from_texts([t.page_content for t in text_chunks], embeddings, index_name=index_name)

In [16]:
from pinecone import Pinecone
from langchain.vectorstores import Pinecone as LangchainPinecone
from pinecone import Pinecone
import os
load_dotenv()

PINECONE_API_KEY = os.environ.get('PINECONE_API_KEY')

# Create an instance of the Pinecone class
pinecone_client = Pinecone(api_key=PINECONE_API_KEY)

# Create an index if it doesn't exist
index_name = "testing"
index = pinecone_client.Index(index_name)

# Initialize embeddings model
embeddings_model = embeddings

# Create Pinecone VectorStore using LangChain
vectorstore = LangchainPinecone(index, embeddings_model.embed_query,"text")

# for doc in text_chunks:
#     vectorstore.add_texts([doc.page_content])

In [1]:
from pinecone import Pinecone
import os
load_dotenv()

PINECONE_API_KEY = os.environ.get('PINECONE_API_KEY')

In [6]:
pinecone_client = Pinecone(api_key=PINECONE_API_KEY)
index = pinecone_client.Index("testing")

q="what is Data science? "
v= embeddings.embed_query(q)

In [7]:
result=index.query(
    vector=v,
    top_k=2,
    include_values=True,
    include_metadata=True
)

In [8]:
cxt=f"{result['matches'][0]['metadata']['text']}  \n  {result['matches'][1]['metadata']['text'] }"

In [9]:
cxt

'in order to verify itscharacteristics.  + MRI—Magnetic resonance imaging is an imaging\ntechnique that uses radiowaves, magnetic fields,and computer analysis to visualize body tissueand structures.\nStereotactic brain needle biopsy —In this proce-\ndure a computer uses information from a CT orMRI to create a three-dimensional map of theoperation site to better guide the needle to per-form the biopsy.due to removal of brain tissue. The resulting scar, left on\nthe brain has the potential to trigger seizures.\nNormal results'

In [10]:
prompt_template="""
Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}
Question: {question}

if the {context} and the {question} not match with each other or far from each other means from the different domains than Must return I don't Know about the {question}
"""

In [11]:
PROMPT=PromptTemplate(template=prompt_template, input_variables=["context", "question"])
chain_type_kwargs={"prompt": PROMPT}

In [12]:
llm=CTransformers(model="D:/Data/GenAI Ineuron/Day12/model/llama-2-7b-chat.ggmlv3.q4_0.bin",
                  model_type="llama",
                  config={'max_new_tokens':512,
                          'temperature':0.8})

In [13]:
from langchain import LLMChain
user_input="what is Data science?"
chain = LLMChain(llm=llm, prompt=PROMPT)
result = chain.run({"context": cxt, "question":user_input})
print(result)


Please provide more context or clarify your question so I can help you better.


In [23]:
# qa=RetrievalQA.from_chain_type(
#     llm=llm, 
#     chain_type="stuff", 
#     retriever=vectorstore.as_retriever(search_kwargs={'k': 2}),
#     return_source_documents=True, 
#     chain_type_kwargs=chain_type_kwargs)


# user_input="what is the Allergy"
# result=qa(user_input)
# print(result)